In [137]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

1. 데이터를 읽어 들임


In [138]:
# 데이터의 경로 지정

path_data_in = './Data_in/'

uriage = pd.read_csv(f'{path_data_in}uriage.csv') # 매출이력 (기간 == 2019 1월~ 2019 7월)

cs = pd.read_excel(f'{path_data_in}kokyaku_daicho.xlsx') # 대리점에서 관리하는 고객 정보

data_dump = pd.read_csv(f'{path_data_in}dump_data.csv')

In [139]:
cs.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   고객이름    200 non-null    object
 1   지역      200 non-null    object
 2   등록일     200 non-null    object
dtypes: object(3)
memory usage: 4.8+ KB


In [140]:
uriage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   purchase_date  2999 non-null   object 
 1   item_name      2999 non-null   object 
 2   item_price     2612 non-null   float64
 3   customer_name  2999 non-null   object 
dtypes: float64(1), object(3)
memory usage: 93.8+ KB


In [141]:
# bool(list(cs['고객이름']) in list(uriage['고객이름']))

In [142]:
# 데이터를 결합

uriage.columns = ['구매날짜', '상품이름', '상품가격', '고객이름']

# df = pd.merge(uriage, cs, how = 'left', on= '고객이름')

# df

2. 데이터의 오류를 살피기


In [143]:
# 입력 오류나, 표기 방법의 차이가 부정합을 일으킴
    # 데이터의 정합성에 문제가 있음

cs.isna().sum()
# uriage.isna().sum() # item_price에서 387개의 결측치 발견
# data_dump.isna().sum()


고객이름    0
지역      0
등록일     0
dtype: int64

In [144]:
uriage.head() # 상품 이름이 통일되지 않음 / 상품 가격, 지역, 등록일에 nan

,구매날짜,상품이름,상품가격,고객이름
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


In [145]:
uriage.sort_values('구매날짜', axis= 0, ignore_index= True, inplace= True)

uriage

,구매날짜,상품이름,상품가격,고객이름
0,2019-01-01 0:20,상품O,1500.0,김민석
1,2019-01-01 12:28,상품S,1900.0,김현준
2,2019-01-01 13:11,상품E,500.0,김은율
3,2019-01-01 15:40,상품Q,1700.0,김우찬
4,2019-01-01 15:47,상품J,1000.0,김동욱
...,...,...,...,...
2994,2019-07-31 22:11,상품r,NaN,김민혁
2995,2019-07-31 23:05,상품L,1200.0,김주혁
2996,2019-07-31 4:22,상품F,600.0,김승재
2997,2019-07-31 4:58,상품 V,2200.0,김윤호


3. 데이터에 오류가 있는 상태로 집계


In [146]:
uriage.info()
uriage.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구매날짜    2999 non-null   object 
 1   상품이름    2999 non-null   object 
 2   상품가격    2612 non-null   float64
 3   고객이름    2999 non-null   object 
dtypes: float64(1), object(3)
memory usage: 93.8+ KB


,구매날짜,상품이름,상품가격,고객이름
0,2019-01-01 0:20,상품O,1500.0,김민석
1,2019-01-01 12:28,상품S,1900.0,김현준
2,2019-01-01 13:11,상품E,500.0,김은율
3,2019-01-01 15:40,상품Q,1700.0,김우찬
4,2019-01-01 15:47,상품J,1000.0,김동욱


In [147]:
# '매출 이력'에서 상품별로 월 매출 합계를 집계
# index == purchase_month(년월만 추출), columns == item_name, size

# 년월만 추출

uriage['구매날짜'] = pd.to_datetime(uriage['구매날짜'], format='%Y-%m-%d %H:%M')

uriage['구매년월'] = uriage['구매날짜'].dt.strftime('%Y-%m')

uriage['구매년월']


0       2019-01
1       2019-01
2       2019-01
3       2019-01
4       2019-01
         ...   
2994    2019-07
2995    2019-07
2996    2019-07
2997    2019-07
2998    2019-07
Name: 구매년월, Length: 2999, dtype: object

In [148]:
# pivot_table 생성

uriage_na = uriage.pivot_table(
    index= '구매년월',
    columns = '상품이름',
    values= '상품가격',
    aggfunc= 'sum'
)

uriage_na

상품이름,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
구매년월,,,,,,,,,,,,,,,,,,,,,
2019-01,1400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1100.0,1200.0,1500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-02,NaN,NaN,NaN,NaN,NaN,NaN,2400.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1900.0,2000.0,2200.0,NaN,NaN
2019-03,NaN,500.0,1300.0,1600.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2300.0,NaN,1700.0,...,NaN,NaN,NaN,NaN,NaN,1900.0,NaN,NaN,NaN,NaN
2019-05,NaN,NaN,NaN,NaN,1900.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2500.0
2019-06,NaN,NaN,NaN,NaN,NaN,2300.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1600.0,NaN,NaN,NaN,NaN,2400.0,NaN
2019-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,1500.0,NaN,1800.0,NaN,NaN,NaN,NaN,NaN


In [149]:
uriage_na = uriage.pivot_table(
    index= '구매년월',
    columns = '상품이름',
    aggfunc= 'size',
    fill_value= 0
)

uriage_na

상품이름,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
구매년월,,,,,,,,,,,,,,,,,,,,,
2019-01,1,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
2019-02,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,1,1,0,0
2019-03,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2019-04,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
2019-05,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2019-06,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2019-07,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


In [150]:
# 매출집계

uriage.pivot_table(
    index= '구매년월',
    columns = '상품이름',
    values= '상품가격',
    aggfunc= 'sum',
    fill_value= 0
)


상품이름,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
구매년월,,,,,,,,,,,,,,,,,,,,,
2019-01,1400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1100.0,1200.0,1500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02,0.0,0.0,0.0,0.0,0.0,0.0,2400.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1900.0,2000.0,2200.0,0.0,0.0
2019-03,0.0,500.0,1300.0,1600.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2300.0,0.0,1700.0,...,0.0,0.0,0.0,0.0,0.0,1900.0,0.0,0.0,0.0,0.0
2019-05,0.0,0.0,0.0,0.0,1900.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2500.0
2019-06,0.0,0.0,0.0,0.0,0.0,2300.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1600.0,0.0,0.0,0.0,0.0,2400.0,0.0
2019-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1500.0,0.0,1800.0,0.0,0.0,0.0,0.0,0.0


4. 상품명의 오류를 수정


In [151]:
# 상품 == A~Z까지 총 26개여야 함
    # 현재 갯수 == 99개
        # 문제점
            # 상품명 뒤의 알파벳의 대/소문자 문제
            # 상품명에 들어가 있는 공백문제

In [152]:
# 모두 대문자로 변경

uriage['상품이름'] =\
    uriage['상품이름'].map(
    lambda x : x.upper()
)

In [153]:
# 공백 제거

uriage['상품이름'] =\
    uriage['상품이름'].map(
    lambda x : x.replace(' ', '')
)

uriage.sort_values(['상품이름'], ignore_index= True, inplace= True)

In [154]:
uriage

,구매날짜,상품이름,상품가격,고객이름,구매년월
0,2019-04-06 11:06:00,상품A,NaN,김태인,2019-04
1,2019-03-12 02:04:00,상품A,100.0,김서윤,2019-03
2,2019-06-03 17:33:00,상품A,100.0,김서진,2019-06
3,2019-01-28 22:13:00,상품A,NaN,김윤호,2019-01
4,2019-03-20 02:16:00,상품A,100.0,김라온,2019-03
...,...,...,...,...,...
2994,2019-06-13 07:18:00,상품Y,2500.0,김민규,2019-06
2995,2019-05-22 17:55:00,상품Y,2500.0,김한결,2019-05
2996,2019-03-03 13:01:00,상품Y,2500.0,김대현,2019-03
2997,2019-06-01 18:57:00,상품Y,2500.0,김서준,2019-06


In [155]:
# 결과 검증

uriage['상품이름'].describe()

count     2999
unique      26
top        상품S
freq       140
Name: 상품이름, dtype: object

5. 금액의 결측치를 수정


In [156]:
# 결측치 확인

uriage.isna().any()

uriage.isna().sum()

# uriage['상품가격'].describe()

구매날짜      0
상품이름      0
상품가격    387
고객이름      0
구매년월      0
dtype: int64

In [157]:
# 특이사항 == 집계 기간에 상품 단가의 변동이 없다는 전제조건이 있음

In [158]:
#  결측치의 조건을 flag로 설정

flag = uriage['상품가격'].isna()


In [159]:
flag

0        True
1       False
2       False
3        True
4       False
        ...  
2994    False
2995    False
2996    False
2997    False
2998    False
Name: 상품가격, Length: 2999, dtype: bool

In [160]:
for idx, trg in enumerate(list(uriage.loc[flag, '상품이름'].unique())): # enumerate() == index와 value를 같이 뽑을 때 사용
    # print(idx, trg) # 상품 이름중 '상품Z'가 출력되지 않은 이유는 '상품Z'에는 결측치가 없기에 loc에서 flag 조건으로 뽑히지 않음
    
    # 상품가격에 결측치가 존재하는 value들을 해당 상품 이름의 최대값으로 치환

        # 상품이름에 결측치가 없는 상품의 행이면서, 해당 상품이름의 가격의 최대값을 price에 저장하고
    
    price = uriage.loc[(~flag)&(uriage['상품이름'] == trg), '상품가격'].max()

        # 상품이름에 결측치가 있는 상품에 price를 집어넣겠다
    uriage['상품가격'].loc[(flag) & (uriage['상품이름'] == trg)] = price



In [161]:
uriage['상품가격'].isna().sum()

0

In [162]:
# 정상적으로 na가 없어졌는 지 확인

for trg in list(uriage['상품이름'].sort_values().unique()):
    print(trg 
          + '의 최고가 : '
          + str(
              uriage.loc[uriage['상품이름'] == trg]['상품가격'].max()
              )
          + '\t'
          + trg 
          + '의 최저가 : ' 
          + str(
              uriage.loc[uriage['상품이름'] == trg]['상품가격'].min(skipna = False)
              )
          )

상품A의 최고가 : 100.0	상품A의 최저가 : 100.0
상품B의 최고가 : 200.0	상품B의 최저가 : 200.0
상품C의 최고가 : 300.0	상품C의 최저가 : 300.0
상품D의 최고가 : 400.0	상품D의 최저가 : 400.0
상품E의 최고가 : 500.0	상품E의 최저가 : 500.0
상품F의 최고가 : 600.0	상품F의 최저가 : 600.0
상품G의 최고가 : 700.0	상품G의 최저가 : 700.0
상품H의 최고가 : 800.0	상품H의 최저가 : 800.0
상품I의 최고가 : 900.0	상품I의 최저가 : 900.0
상품J의 최고가 : 1000.0	상품J의 최저가 : 1000.0
상품K의 최고가 : 1100.0	상품K의 최저가 : 1100.0
상품L의 최고가 : 1200.0	상품L의 최저가 : 1200.0
상품M의 최고가 : 1300.0	상품M의 최저가 : 1300.0
상품N의 최고가 : 1400.0	상품N의 최저가 : 1400.0
상품O의 최고가 : 1500.0	상품O의 최저가 : 1500.0
상품P의 최고가 : 1600.0	상품P의 최저가 : 1600.0
상품Q의 최고가 : 1700.0	상품Q의 최저가 : 1700.0
상품R의 최고가 : 1800.0	상품R의 최저가 : 1800.0
상품S의 최고가 : 1900.0	상품S의 최저가 : 1900.0
상품T의 최고가 : 2000.0	상품T의 최저가 : 2000.0
상품U의 최고가 : 2100.0	상품U의 최저가 : 2100.0
상품V의 최고가 : 2200.0	상품V의 최저가 : 2200.0
상품W의 최고가 : 2300.0	상품W의 최저가 : 2300.0
상품X의 최고가 : 2400.0	상품X의 최저가 : 2400.0
상품Y의 최고가 : 2500.0	상품Y의 최저가 : 2500.0
상품Z의 최고가 : 2600.0	상품Z의 최저가 : 2600.0


6. 고객 이름의 오류를 수정


In [163]:
uriage['고객이름'].head()

0    김태인
1    김서윤
2    김서진
3    김윤호
4    김라온
Name: 고객이름, dtype: object

In [164]:
cs['고객이름'].head()

0    김 현성
1    김 도윤
2    김 지한
3    김 하윤
4    김 시온
Name: 고객이름, dtype: object

In [165]:
# 공백을 제거

cs['고객이름'] = cs['고객이름'].map(
    lambda x : x.replace(' ', '')
)

7. 날짜 오류를 수정


In [166]:
cs['등록일']

0      2018-01-04 00:00:00
1                    42782
2      2018-01-07 00:00:00
3                    42872
4                    43127
              ...         
195    2017-06-20 00:00:00
196    2018-06-20 00:00:00
197    2017-04-29 00:00:00
198    2019-04-19 00:00:00
199    2019-04-23 00:00:00
Name: 등록일, Length: 200, dtype: object

In [167]:
# 문자 데이터들만을 추출

flag_str = cs['등록일'].astype('str').str.isdigit() # 숫자 모양의 문자 데이터들에 대한 flag
flag_str.sum() # 갯수 확인

22

In [170]:
# float로 변환된 날짜를 일 단위로 변환해 치환하고, 기준일을 1900/01/01로 표현

fromSerial = pd.to_timedelta(cs.loc[flag_str, '등록일'].astype('float'), unit= 'D') + pd.to_datetime('1900/01/01') # unit 매개변수는 입력의 단위를 의미, 'D'는 일을 기준으로 입력한다는 의미
                                        # timedelta로 22개의 값들의 타입을 float로 변환 후 이를 D, 즉 이 숫자들의 기준을 '일'로 만들어 시계열 데이터화 하겠다는 구문 + 년월 더하기

fromSerial # 위의 22개의 데이터가 치환됨

1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 등록일, dtype: datetime64[ns]

In [171]:
# cs['등록일']에서 숫자가 아닌 시계열로 존재했던 데이터들을 %Y-%m-%d로 변환 후 치환

fromString = pd.to_datetime(cs.loc[~flag_str, '등록일'])

fromString

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 178, dtype: datetime64[ns]

In [172]:
# 만들어진 fromSerial과 fromString을 유니언 결합

cs['등록일'] = pd.concat([fromSerial, fromString]) # fromSerial에서 원본의 index 값을 그대로 들고 있기에 이렇게 가능

cs

,고객이름,지역,등록일
0,김현성,H시,2018-01-04
1,김도윤,E시,2017-02-18
2,김지한,A시,2018-01-07
3,김하윤,F시,2017-05-19
4,김시온,E시,2018-01-29
...,...,...,...
195,김재희,G시,2017-06-20
196,김도영,E시,2018-06-20
197,김이안,F시,2017-04-29
198,김시현,H시,2019-04-19


In [174]:
# 등록일에서 연,월을 추출하여 등록년월 파생변수 생성

cs['등록년월'] = cs['등록일'].dt.strftime('%Y%m')


In [176]:
# 등록년월별 고객의 수를 세보자

rslt = cs.groupby('등록년월')['고객이름'].count()

rslt

등록년월
201701    15
201702    11
201703    14
201704    15
201705    13
201706    14
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 고객이름, dtype: int64

In [187]:
# 등록일 column에 숫자 데이터가 남아있는지 확인

'''cs['등록일'].map(
    lambda x : x.isdigit()
)''' # 오류발생 == 다 시계열 데이터기 때문, 근데 이건 하나만 시계열이여도 되는거 아냐 ?

flag_str2 = cs['등록일'].astype('str').str.isdigit()

flag_str2.sum()

0

8. 고객 이름을 키로 두 개의 데이터를 결합(조인)


In [188]:
df = pd.merge(uriage, cs, how = 'left', on = '고객이름')

df

,구매날짜,상품이름,상품가격,고객이름,구매년월,지역,등록일,등록년월
0,2019-04-06 11:06:00,상품A,100.0,김태인,2019-04,F시,2017-03-12,201703
1,2019-03-12 02:04:00,상품A,100.0,김서윤,2019-03,G시,2017-01-30,201701
2,2019-06-03 17:33:00,상품A,100.0,김서진,2019-06,G시,2018-06-29,201806
3,2019-01-28 22:13:00,상품A,100.0,김윤호,2019-01,G시,2017-02-05,201702
4,2019-03-20 02:16:00,상품A,100.0,김라온,2019-03,F시,2018-07-29,201807
...,...,...,...,...,...,...,...,...
2994,2019-06-13 07:18:00,상품Y,2500.0,김민규,2019-06,B시,2017-06-13,201706
2995,2019-05-22 17:55:00,상품Y,2500.0,김한결,2019-05,C시,2017-02-18,201702
2996,2019-03-03 13:01:00,상품Y,2500.0,김대현,2019-03,H시,2017-01-30,201701
2997,2019-06-01 18:57:00,상품Y,2500.0,김서준,2019-06,C시,2017-06-17,201706


9. 정제한 데이터를 덤프


In [ ]:
# 어떤 데이터 순서로 sorting해서 저장해야 할 지에 대한 고민
path_data_out = './Data_out/'

# 데이터를 sorting


# 데이터를 저장
df.save(f'{path_data_out}dump_data.csv')

10. 데이터를 집계